---

Data Mining Assignment 03.

Due Date: 5 Ordibehesht 1395, before 10:00 AM.

Rename the file so that it contains the names of the members of the group.

Email the .ipynb file to "taherian.khu@gmail.com".

Do not forget to choose DM_Assignment03_names as email subject

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 3 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) 

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    text_file = open('university_towns.txt', 'r')
    lines = text_file.readlines()
    laststate='';ln=[];df = pd.DataFrame(columns =["State", "RegionName"])
    for l in lines:
        if '[edit]' in l:
            laststate=(l.split('[')[0]).strip()
        else:
            df.loc[len(df)]=[laststate,(l.split('(')[0]).strip()]
    return df
get_list_of_university_towns().head(6)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy


In [28]:
global gdp
gdp=pd.read_excel('gdplev.xls', skiprows = 7, parse_cols=[4, 5, 6], names=['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars'])
gdp = gdp.set_index('Quarter')

In [55]:
rec_start = []
rec_end = []
rec_bottom = []
recession = False
rec_min = (0, 0)
for i, row in enumerate(gdp.values):
    if not recession and (gdp.iloc[i-1, 0] > row[0] > gdp.iloc[i+1, 0]):
        recession = True
        rec_min = (i+1, gdp.iloc[i+1, 0])
        rec_start.append(gdp.index[i])
    elif recession and (gdp.iloc[i-1, 0] < row[0] < gdp.iloc[i+1, 0]):
        recession = False
        mini, minval = rec_min
        if (minval > gdp.iloc[i-1, 0]):
            rec_min = (i-1, gdp.iloc[i-1, 0])
        mini, minval = rec_min
        rec_bottom.append(gdp.index[mini])
        rec_end.append(gdp.index[i])
    elif recession:
        mini, minval = rec_min
        if (minval > gdp.iloc[i, 0]):
            rec_min = (i, gdp.iloc[i, 0])

In [50]:
def get_recession_start():
#     '''Returns the year and quarter of the recession start time
    return rec_start
get_recession_start()

['1949q1', '1953q3', '1957q4', '2008q4']

In [51]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    return rec_end
get_recession_end()

['1950q1', '1954q2', '1958q2', '2009q3']

In [57]:
def get_recession_bottom():
# recession bottom time 
    return rec_bottom
get_recession_bottom()

['1949q4', '1954q1', '1958q1', '2009q2']

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"